# Note book for the project 2 

Kaggle competition link: [Submition]('https://www.kaggle.com/c/epfml17-text/submit')

Importing usefull library

In [8]:
import numpy as np
import pandas as pd
%matplotlib inline
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle
import random

In [3]:
ls

Project2.ipynb                     pickle_vocab.py
README.md                          sample_submission.csv
UntouchedData/                     test_data.txt
build_vocab.sh                     train_neg.txt
cooc.pkl                           train_neg_full.txt
cooc.py                            train_pos.txt
cut_vocab.sh                       train_pos_full.txt
embeddings.npy                     vocab.pkl
glove_solution_WILL_COME_LATER.py  vocab.txt
glove_template.py                  vocab_cut.txt


In [22]:
#pkl = pd.read_pickle("cooc.pkl")
pkl.toarray().shape

(21161, 21161)

In [12]:
print("loading cooccurrence matrix")
with open('cooc.pkl', 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

nmax = 100
print("using nmax =", nmax, ", cooc.max() =", cooc.max())
print("initializing embeddings")
embedding_dim = 20
xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
eta = 0.001
alpha = 3 / 4
epochs = 10

for epoch in range(epochs):
    print("epoch {}".format(epoch))
    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
        # fill in your SGD code here, 
        # for the update resulting from co-occurence (i,j)
        
        np.save('embeddings', xs)


loading cooccurrence matrix
6496907 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0


KeyboardInterrupt: 

In [18]:
type(cooc.data)

numpy.ndarray